In [ ]:
%matplotlib

In [ ]:
import os, sys
import keras
import pickle
import talos
import numpy as np
from numpy.random import seed

from Trajectories.UniformDist import UniformDist
from Trajectories.DHGridDist import DHGridDist
from Trajectories.RandomDist import RandomDist
from CelestialBodies.Planets import Earth
from GravityModels.SphericalHarmonics import SphericalHarmonics
from Preprocessors.MinMaxTransform import MinMaxTransform
from Support.transformations import sphere2cart, cart2sph, project_acceleration
from GravityModels.NNSupport.NN_hyperparam import NN_hyperparam

from GravityModels.NN_Base import NN_Base
from GravityModels.NNSupport.NN_DeepLayers import *
from GravityModels.NNSupport.NN_MultiLayers import *
from GravityModels.NNSupport.NN_SingleLayers import *

from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import SGD, Adadelta, Nadam, Adam, RMSprop
seed(1)

from talos.utils.recover_best_model import recover_best_model
from talos import Analyze, Evaluate, Scan, Predict, Deploy, Restore
import matplotlib.pyplot as plt

# Search through Hyperparameter Space

In [ ]:
planet = Earth()
point_count = 1000
#trajectory = UniformDist(planet, planet.radius, 10000)
trajectory = RandomDist(planet, [planet.radius, planet.radius*1.1], point_count)

gravityModel = SphericalHarmonics(planet.sh_file, degree=None, trajectory=trajectory)
gravityModel.load() 

pos_sphere = cart2sph(trajectory.positions)
acc_proj = project_acceleration(pos_sphere, gravityModel.accelerations)

preprocessor = MinMaxTransform()
preprocessor.percentTest = 0.01
preprocessor.split(pos_sphere, acc_proj)
preprocessor.fit()
x_train, x_test, y_train, y_test = preprocessor.apply_transform()

p = {'first_neuron':[128, 64, 16],
        'hidden_layers':[1, 3],
        'batch_size': [5, 15],
        'epochs': [100, 200, 300],
        'dropout': [0.0, 0.2],
        'lr' : [0.1, 0.15, 0.2],
        'kernel_initializer': ['normal'],
        'optimizer': [Nadam, Adadelta],
        'shapes' : ['brick'], 
        'losses': ['mean_squared_error', 'mean_absolute_error'],
        'activation':['relu', 'elu', 'tanh']
        }

t = talos.Scan(x=x_train,
                            y=y_train,
                            model=NN_hyperparam,
                            params=p,
                            experiment_name='Initial_Search',
                            fraction_limit=0.005)
                            
d = Deploy(t, "third_iteration", 'val_accuracy', asc=False)


# Load the Scan Object and Analyze

In [65]:
excludes = ['shape', 'epochs', 'round_epochs']
excludes = ['accuracy', 'loss', 'val_loss', 'round_epochs', 'epochs']
first = '../Initial_Search/070620135028.csv'
second = '/Users/johnmartin/Documents/GraduateSchool/Research/GravityML/nnPosAcc/Initial_Search/070720072710.csv'
current = second
a = Analyze(current)

In [ ]:
a.plot_corr('val_accuracy', ['acc', 'loss', 'val_accuracy', 'val_loss', 'round_epochs'])

In [ ]:
#a.plot_hist('val_accuracy', bins=20)
a.plot_bars('lr','val_accuracy', 'kernel_initializer','first_neuron')
a.plot_box('lr', 'val_accuracy', 'optimizer')

In [ ]:
best = a.best_params('val_accuracy', excludes)
train_high = a.high('accuracy')
val_high = a.high("val_accuracy")

# Evaluate Scan Object on Test Data

In [66]:
planet = Earth()
point_count = 1000
#trajectory = UniformDist(planet, planet.radius, 10000)
trajectory = RandomDist(planet, [planet.radius, planet.radius*1.1], point_count)

gravityModel = SphericalHarmonics(planet.sh_file, degree=None, trajectory=trajectory)
gravityModel.load() 

pos_sphere = cart2sph(trajectory.positions)
acc_proj = project_acceleration(pos_sphere, gravityModel.accelerations)

density_deg = 175
map_grid = DHGridDist(planet, planet.radius, degree=density_deg)

gravityModelMap = SphericalHarmonics(planet.sh_file, degree=None, trajectory=map_grid)
gravityModelMap.load() 

test_pos_sphere = cart2sph(map_grid.positions)
test_acc_proj = project_acceleration(test_pos_sphere, gravityModelMap.accelerations)

The total number of points is not a perfect square
The total number of points changed to 961
Found existing acceleration.data at /Users/johnmartin/Documents/GraduateSchool/Research/GravityML/nnPosAcc/Trajectories/TrajectoryBase/../../Files/Trajectories/RandomDist/earthN_961_RadBounds[6378136.6, 7015950.26]/SphericalHarmonics_GGM03S_None/
Found existing acceleration.data at /Users/johnmartin/Documents/GraduateSchool/Research/GravityML/nnPosAcc/Trajectories/TrajectoryBase/../../Files/Trajectories/DHGridDist/earth_Deg175_Rad6378136.6/SphericalHarmonics_GGM03S_None/


In [67]:
preprocessor = MinMaxTransform()
preprocessor.percentTest = 0.01
preprocessor.split(pos_sphere, acc_proj)
preprocessor.fit()
x_train, x_test, y_train, y_test = preprocessor.apply_transform()
x_test, y_test = preprocessor.apply_transform(test_pos_sphere, test_acc_proj)

In [71]:
#a = Analyze(t) # Reporting() and Analyze() are the same
p = Predict(t)
prediction = p.predict(x_test,'val_accuracy', asc=False)
error_inst = abs(np.divide((prediction - y_test),y_test)*100)
print(np.average(error_inst))
print(np.median(error_inst))

# Pick the best model and evaluate the average absolute difference
#e = Evaluate(t)
#error = e.evaluate(x_test,y_test, metric='val_accuracy', task='continuous', asc=False)

# Load a prior model if interested. 
#r = Restore('/Users/johnmartin/Documents/GraduateSchool/Research/GravityML/nnPosAcc/Notebooks/current_model.zip')
# r.results
# r.details
# r.model #keras sequential model
# r.model.predict(x_test)

673.1206732720145
30.76568083736315


In [72]:
error_inst = error_inst.squeeze()
